In [14]:
import pickle
import pandas as pd
import json
from tqdm import tqdm
import seaborn as sns
sns.set()

In [4]:
wmt16_sys = pickle.load(open('../../wmt16_sys_metadata.pkl', 'rb'))
wmt16_sys.head(1)

,lp,HITId,WorkerId,score,time,system,type,sid,reference,source,output
0,en-ru,3XJOUITW8URT45XA3297SQ9SRI6QT9,A0447,79,1661,PROMT-Rule-based,SYSTEM,52,В те годы он пользовался успехом у женщин.\n,He was quite the heartthrob back then.\n,Он был вполне сердцеедом тогда.\n


In [45]:
df = wmt16_sys[wmt16_sys.lp.str.endswith('en')].copy()

### Export

In [42]:
centering = {}

for i, group in df.groupby('WorkerId', as_index=False):
    mean = group.mean()['score']
    std = group.std()['score']
    centering[i] = (mean, std)

In [58]:
df['norm_score'] = df.apply(lambda x: (x['score'] - centering[x['WorkerId']][0]) / centering[x['WorkerId']][1] , axis=1)
df.head(1)

,lp,HITId,WorkerId,score,time,system,type,sid,reference,source,output,norm_score
12300,de-en,3QQUBC64ZEEFIPT2SKINDZZTZ3SNXN,A0077,100,2052,online-A,REF,241,This would allow for appeals to be made agains...,"Sie würde es erlauben, gegen Urteile Einspruch...",It would allow it to appeal against judgments.\n,0.84735


In [52]:
shuf_df = df.sample(len(df), random_state=119)
print(len(df))

In [57]:
split = int(len(df) * 0.8)
fh = open('./train.jsonl', 'wt')

for i, row in tqdm(shuf_df[:split].iterrows(), total=split):
    ex = {}
    ex['candidate'] = row['output']
    ex['reference'] = row['reference']
    ex['score'] = row['norm_score']
    
    fh.write(json.dumps(ex, ensure_ascii=False) + '\n')

fh = open('./valid.jsonl', 'wt')
for i, row in tqdm(shuf_df[:split].iterrows(), total=split):
    ex = {}
    ex['candidate'] = row['output']
    ex['reference'] = row['reference']
    ex['score'] = row['norm_score']
    
    fh.write(json.dumps(ex, ensure_ascii=False) + '\n')

 80%|████████  | 152080/190100 [00:28<00:07, 5318.94it/s]
